### Extracting companies from each directories

### Web Crawling on directories

In [ ]:
"""
============================================
Enhanced Zimbabwe Company Directory Scraper
============================================
Features:
- Breadth-First Search (BFS) algorithm
- Progressive checkpoint saving
- Performance statistics & visualization
- Network graph visualization
- City and description extraction
- Internet interruption resilient
- Publication-ready analytics

Author: Enhanced Version
Date: 2025
============================================
"""

import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import deque, defaultdict
from dataclasses import dataclass, asdict
from typing import Set, Dict, List, Optional, Tuple
import re
import time
import random
import json
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime, timedelta

# ============================================
# CONFIGURATION
# ============================================

class Config:
    """Centralized configuration"""
    # User agents for rotation
    USER_AGENTS = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) AppleWebKit/605.1.15 Version/17.0 Safari/605.1.15",
    ]

    # Request settings
    REQUEST_TIMEOUT = 15
    SLEEP_MIN, SLEEP_MAX = 0.3, 0.8  # Faster but respectful
    RETRY_ATTEMPTS = 3

    # Crawl limits (preventing infinite loops)
    MAX_PAGES_PER_SEED = 3800          # Maximum pages per initial URL
    MAX_DEPTH = 1                     # Maximum BFS depth
    MAX_DETAIL_PAGES = 2000            # Maximum detail pages per seed
    MAX_CATEGORIES = 800              # Maximum categories to explore

    # ZimbabweYP specific settings
    ZYP_MAX_PAGES_PER_CATEGORY = 100  # Reduced for speed
    ZYP_DETAIL_WORKERS = 6            # Parallel detail fetching

    # Checkpoint settings
    CHECKPOINT_INTERVAL = 50          # Save every N companies
    CHECKPOINT_DIR = Path("checkpoints")
    OUTPUT_DIR = Path("output")
    STATS_DIR = Path("statistics")

    # Country
    COUNTRY = "Zimbabwe"

# Create directories
for dir_path in [Config.CHECKPOINT_DIR, Config.OUTPUT_DIR, Config.STATS_DIR]:
    dir_path.mkdir(exist_ok=True)

# ============================================
# DATA STRUCTURES
# ============================================

@dataclass
class Company:
    """Company data structure"""
    name: str = ""
    address: str = ""
    city: str = ""
    phone: str = ""
    email: str = ""
    website: str = ""
    description: str = ""
    source_url: str = ""
    country: str = Config.COUNTRY
    crawl_timestamp: str = ""

@dataclass
class CrawlStats:
    """Statistics for a crawl session"""
    seed_url: str
    start_time: float
    end_time: float = 0
    pages_crawled: int = 0
    detail_pages_crawled: int = 0
    companies_found: int = 0
    errors: int = 0
    avg_response_time: float = 0
    success_rate: float = 0

    def duration(self) -> float:
        return self.end_time - self.start_time if self.end_time else time.time() - self.start_time

    def pages_per_minute(self) -> float:
        duration_min = self.duration() / 60
        return self.pages_crawled / duration_min if duration_min > 0 else 0

# ============================================
# UTILITIES
# ============================================

class SessionManager:
    """Manages HTTP sessions with retry logic"""

    @staticmethod
    def create_session() -> requests.Session:
        session = requests.Session()
        retry = Retry(
            total=Config.RETRY_ATTEMPTS,
            backoff_factor=0.3,
            status_forcelist=[429, 500, 502, 503, 504],
        )
        adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=20)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        return session

    @staticmethod
    def get_headers() -> Dict[str, str]:
        return {
            "User-Agent": random.choice(Config.USER_AGENTS),
            "Accept-Language": "en-US,en;q=0.9",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Referer": "https://www.google.com/",
        }

class TextProcessor:
    """Text cleaning and extraction utilities"""

    PHONE_RE = re.compile(r"(?:\+?\d[\d\-\s\(\)]{6,}\d)")
    EMAIL_RE = re.compile(r"[A-Z0-9._%+\-]+@[A-Z0-9.\-]+\.[A-Z]{2,}", re.I)

    # Zimbabwe city patterns
    CITIES = [
        "Harare", "Bulawayo", "Chitungwiza", "Mutare", "Gweru", "Kwekwe",
        "Kadoma", "Masvingo", "Chinhoyi", "Marondera", "Norton", "Chegutu",
        "Bindura", "Beitbridge", "Redcliff", "Victoria Falls", "Hwange",
        "Rusape", "Chiredzi", "Kariba", "Karoi", "Zvishavane"
    ]

    @staticmethod
    def clean(text) -> str:
        if not text:
            return ""
        return re.sub(r"\s+", " ", str(text)).strip()

    @staticmethod
    def extract_phones(text: str) -> List[str]:
        return list({TextProcessor.clean(m.group(0)) for m in TextProcessor.PHONE_RE.finditer(text or "")})

    @staticmethod
    def extract_emails(text: str) -> List[str]:
        return list({TextProcessor.clean(m.group(0)) for m in TextProcessor.EMAIL_RE.finditer(text or "")})

    @staticmethod
    def extract_city(text: str) -> str:
        """Extract city name from address/text"""
        if not text:
            return ""
        text_upper = text.upper()
        for city in TextProcessor.CITIES:
            if city.upper() in text_upper:
                return city
        return ""

class URLUtils:
    """URL manipulation utilities"""

    @staticmethod
    def get_domain(url: str) -> str:
        try:
            return urlparse(url).netloc.lower().replace("www.", "")
        except:
            return ""

    @staticmethod
    def same_domain(url1: str, url2: str) -> bool:
        return URLUtils.get_domain(url1) == URLUtils.get_domain(url2)

    @staticmethod
    def normalize(url: str) -> str:
        try:
            p = urlparse(url)
            return f"{p.scheme}://{p.netloc}{p.path or '/'}"
        except:
            return url

    @staticmethod
    def is_zimbabwe_yp(url: str) -> bool:
        return "zimbabweyp.com" in URLUtils.get_domain(url)

def sleep_random():
    """Random sleep to avoid rate limiting"""
    time.sleep(random.uniform(Config.SLEEP_MIN, Config.SLEEP_MAX))

# ============================================
# HTTP & PARSING
# ============================================

class HTTPClient:
    """HTTP client with timing and error tracking"""

    def __init__(self):
        self.session = SessionManager.create_session()
        self.response_times = []
        self.errors = 0
        self.success = 0

    def get_soup(self, url: str) -> Optional[BeautifulSoup]:
        """Fetch URL and return BeautifulSoup object"""
        start = time.time()
        try:
            resp = self.session.get(
                url,
                headers=SessionManager.get_headers(),
                timeout=Config.REQUEST_TIMEOUT,
                allow_redirects=True
            )

            if resp.status_code >= 400:
                self.errors += 1
                return None

            if not resp.encoding:
                resp.encoding = "utf-8"

            self.response_times.append(time.time() - start)
            self.success += 1
            return BeautifulSoup(resp.text, "lxml")

        except Exception as e:
            self.errors += 1
            return None

    def avg_response_time(self) -> float:
        return sum(self.response_times) / len(self.response_times) if self.response_times else 0

    def success_rate(self) -> float:
        total = self.success + self.errors
        return (self.success / total * 100) if total > 0 else 0

# ============================================
# EXTRACTION LOGIC
# ============================================

class CompanyExtractor:
    """Extract company information from pages"""

    ORG_TYPES = {"Organization", "LocalBusiness", "Corporation", "Company", "NGO", "Store"}

    @staticmethod
    def from_json_ld(soup: BeautifulSoup, source_url: str) -> List[Company]:
        """Extract companies from JSON-LD structured data"""
        companies = []

        for tag in soup.find_all("script", type=lambda t: t and "ld+json" in t):
            try:
                data = json.loads(tag.string or "")
            except:
                continue

            items = []
            if isinstance(data, list):
                items = data
            elif isinstance(data, dict):
                if "@graph" in data:
                    items = data["@graph"]
                else:
                    items = [data]

            for item in items:
                if not isinstance(item, dict):
                    continue

                item_type = item.get("@type", "")
                types = {item_type} if isinstance(item_type, str) else set(item_type or [])

                if not types.intersection(CompanyExtractor.ORG_TYPES):
                    continue

                name = TextProcessor.clean(item.get("name", ""))
                if not name:
                    continue

                # Parse address
                addr = item.get("address", "")
                if isinstance(addr, dict):
                    addr_parts = [
                        addr.get("streetAddress", ""),
                        addr.get("addressLocality", ""),
                        addr.get("addressRegion", ""),
                        addr.get("postalCode", ""),
                    ]
                    address = TextProcessor.clean(" ".join(p for p in addr_parts if p))
                else:
                    address = TextProcessor.clean(addr)

                company = Company(
                    name=name,
                    address=address,
                    city=TextProcessor.extract_city(address),
                    phone=TextProcessor.clean(item.get("telephone", "")),
                    email=TextProcessor.clean(item.get("email", "")),
                    website=TextProcessor.clean(item.get("url", "")),
                    description=TextProcessor.clean(item.get("description", "")),
                    source_url=source_url,
                    crawl_timestamp=datetime.now().isoformat()
                )
                companies.append(company)

        return companies

    @staticmethod
    def from_listing_blocks(soup: BeautifulSoup, base_url: str) -> Tuple[List[Company], List[str]]:
        """Extract companies and detail links from listing blocks"""
        companies = []
        detail_links = set()

        # Common listing selectors
        selectors = [
            'div[class*="listing"]', 'div[class*="result"]', 'div[class*="company"]',
            'div[class*="business"]', 'li[class*="listing"]', 'article', '.card'
        ]

        blocks = []
        for sel in selectors:
            blocks.extend(soup.select(sel))

        for block in blocks[:100]:  # Limit to avoid over-processing
            try:
                # Extract name
                name_elem = (
                    block.select_one('h2 a, h3 a, h4 a, .name a, .title a') or
                    block.select_one('h2, h3, h4, .name, .title, strong')
                )
                name = TextProcessor.clean(name_elem.get_text()) if name_elem else ""

                if not name:
                    continue

                # Extract address
                addr_elem = block.select_one('address, .address, .location, [itemprop="address"]')
                address = TextProcessor.clean(addr_elem.get_text()) if addr_elem else ""

                # Extract contact info
                phone_elem = block.select_one('a[href^="tel:"], .phone, [itemprop="telephone"]')
                phone = TextProcessor.clean(phone_elem.get_text()) if phone_elem else ""

                email_elem = block.select_one('a[href^="mailto:"], .email')
                email = TextProcessor.clean(email_elem.get_text()) if email_elem else ""

                # Extract description
                desc_elem = block.select_one('.description, .about, .summary, p')
                description = TextProcessor.clean(desc_elem.get_text()[:500]) if desc_elem else ""

                # Find links
                website = ""
                detail_url = None

                for a in block.find_all("a", href=True):
                    href = a["href"]
                    if href.startswith(("mailto:", "tel:", "javascript:", "#")):
                        continue

                    full_url = urljoin(base_url, href)

                    if not URLUtils.same_domain(full_url, base_url):
                        website = full_url
                    elif not detail_url:
                        detail_url = full_url

                # Fallback: extract from block text
                block_text = block.get_text()
                if not phone:
                    phones = TextProcessor.extract_phones(block_text)
                    phone = phones[0] if phones else ""
                if not email:
                    emails = TextProcessor.extract_emails(block_text)
                    email = emails[0] if emails else ""

                company = Company(
                    name=name,
                    address=address,
                    city=TextProcessor.extract_city(address),
                    phone=phone,
                    email=email,
                    website=website,
                    description=description,
                    source_url=detail_url or base_url,
                    crawl_timestamp=datetime.now().isoformat()
                )
                companies.append(company)

                if detail_url:
                    detail_links.add(detail_url)

            except Exception:
                continue

        return companies, list(detail_links)

    @staticmethod
    def from_detail_page(soup: BeautifulSoup, url: str) -> Optional[Company]:
        """Extract company from detail page"""
        # Try JSON-LD first
        jsonld_companies = CompanyExtractor.from_json_ld(soup, url)
        if jsonld_companies:
            return jsonld_companies[0]

        # Heuristic extraction
        name_elem = soup.select_one('h1, .company-name, .page-title, [itemprop="name"]')
        name = TextProcessor.clean(name_elem.get_text()) if name_elem else ""

        if not name:
            title = soup.find("title")
            name = TextProcessor.clean(title.get_text()) if title else ""

        addr_elem = soup.select_one('address, .address, .location, [itemprop="address"]')
        address = TextProcessor.clean(addr_elem.get_text()) if addr_elem else ""

        # Extract description/about
        desc_elem = soup.select_one('.about, .description, .company-info, [itemprop="description"]')
        description = ""
        if desc_elem:
            description = TextProcessor.clean(desc_elem.get_text()[:1000])

        # Extract from full text
        page_text = soup.get_text()
        phones = TextProcessor.extract_phones(page_text)
        emails = TextProcessor.extract_emails(page_text)

        # Find website
        website = ""
        for a in soup.find_all("a", href=True):
            text = a.get_text().lower()
            if "website" in text or "visit" in text:
                website = urljoin(url, a["href"])
                break

        return Company(
            name=name,
            address=address,
            city=TextProcessor.extract_city(address),
            phone=phones[0] if phones else "",
            email=emails[0] if emails else "",
            website=website,
            description=description,
            source_url=url,
            crawl_timestamp=datetime.now().isoformat()
        ) if name else None

# ============================================
# PAGINATION
# ============================================

class PaginationHandler:
    """Handle pagination across different sites"""

    @staticmethod
    def find_next_page(soup: BeautifulSoup, current_url: str) -> Optional[str]:
        """Find the next page URL"""
        # Check rel="next"
        for a in soup.find_all("a", href=True, rel=True):
            rels = a.get("rel")
            if isinstance(rels, str):
                rels = [rels]
            if any("next" in r.lower() for r in (rels or [])):
                return urljoin(current_url, a["href"])

        # Check text/aria labels
        for a in soup.find_all("a", href=True):
            text = a.get_text().lower()
            classes = " ".join(a.get("class", [])).lower()
            aria = (a.get("aria-label") or "").lower()

            if any(term in text for term in ["next", "»", ">"]) or "next" in classes or "next" in aria:
                return urljoin(current_url, a["href"])

        # Check URL patterns
        # Pattern: ?page=N
        match = re.search(r"([?&])(page|p)=(\d+)", current_url, re.I)
        if match:
            next_num = int(match.group(3)) + 1
            return re.sub(r"([?&])(page|p)=\d+", rf"\1\2={next_num}", current_url)

        # Pattern: /page/N/
        match = re.search(r"/page/(\d+)/?$", current_url, re.I)
        if match:
            next_num = int(match.group(1)) + 1
            return re.sub(r"/page/\d+/?$", f"/page/{next_num}/", current_url)

        return None

# ============================================
# BFS CRAWLER
# ============================================

class BFSCrawler:
    """Breadth-First Search web crawler with checkpointing"""

    def __init__(self, seed_url: str, checkpoint_id: str):
        self.seed_url = seed_url
        self.checkpoint_id = checkpoint_id
        self.domain = URLUtils.get_domain(seed_url)

        # BFS data structures
        self.queue = deque([(seed_url, 0)])  # (url, depth)
        self.visited_listing = set()
        self.visited_detail = set()

        # Results
        self.companies: List[Company] = []
        self.stats = CrawlStats(seed_url=seed_url, start_time=time.time())

        # Network graph for visualization
        self.graph = nx.DiGraph()
        self.graph.add_node(seed_url, type="seed", depth=0)

        # HTTP client
        self.client = HTTPClient()

        # Load checkpoint if exists
        self._load_checkpoint()

    def _load_checkpoint(self):
        """Load progress from checkpoint"""
        checkpoint_file = Config.CHECKPOINT_DIR / f"{self.checkpoint_id}.pkl"
        if checkpoint_file.exists():
            try:
                with open(checkpoint_file, "rb") as f:
                    data = pickle.load(f)
                    self.queue = data["queue"]
                    self.visited_listing = data["visited_listing"]
                    self.visited_detail = data["visited_detail"]
                    self.companies = data["companies"]
                    self.stats = data["stats"]
                    self.graph = data["graph"]
                print(f"✓ Resumed from checkpoint: {len(self.companies)} companies, {len(self.visited_listing)} pages")
            except Exception as e:
                print(f"⚠ Could not load checkpoint: {e}")

    def _save_checkpoint(self):
        """Save progress to checkpoint"""
        checkpoint_file = Config.CHECKPOINT_DIR / f"{self.checkpoint_id}.pkl"
        try:
            with open(checkpoint_file, "wb") as f:
                pickle.dump({
                    "queue": self.queue,
                    "visited_listing": self.visited_listing,
                    "visited_detail": self.visited_detail,
                    "companies": self.companies,
                    "stats": self.stats,
                    "graph": self.graph,
                }, f)
        except Exception as e:
            print(f"⚠ Checkpoint save failed: {e}")

    def crawl(self) -> Tuple[List[Company], CrawlStats, nx.DiGraph]:
        """Execute BFS crawl"""
        print(f"\n{'='*60}")
        print(f"Starting BFS crawl: {self.seed_url}")
        print(f"Domain: {self.domain}")
        print(f"{'='*60}\n")

        pages_crawled = 0
        last_checkpoint = 0

        while self.queue and pages_crawled < Config.MAX_PAGES_PER_SEED:
            url, depth = self.queue.popleft()

            # Skip if already visited or too deep
            if url in self.visited_listing or depth > Config.MAX_DEPTH:
                continue

            self.visited_listing.add(url)

            # Fetch page
            soup = self.client.get_soup(url)
            if not soup:
                self.stats.errors += 1
                continue

            pages_crawled += 1
            self.stats.pages_crawled += 1

            # Extract companies from this page
            jsonld_companies = CompanyExtractor.from_json_ld(soup, url)
            block_companies, detail_links = CompanyExtractor.from_listing_blocks(soup, url)

            self.companies.extend(jsonld_companies)
            self.companies.extend(block_companies)
            self.stats.companies_found = len(self.companies)

            # Add detail links to graph and queue
            for detail_url in detail_links[:Config.MAX_DETAIL_PAGES]:
                if detail_url not in self.visited_detail:
                    self.graph.add_edge(url, detail_url, type="detail")
                    self.visited_detail.add(detail_url)

            # Discover new listing pages (same domain only)
            for a in soup.find_all("a", href=True):
                href = a["href"]
                if href.startswith(("mailto:", "tel:", "javascript:", "#")):
                    continue

                full_url = urljoin(url, href)

                if (URLUtils.same_domain(full_url, self.seed_url) and
                    full_url not in self.visited_listing and
                    depth + 1 <= Config.MAX_DEPTH):

                    self.queue.append((full_url, depth + 1))
                    self.graph.add_edge(url, full_url, type="listing")
                    self.graph.add_node(full_url, type="listing", depth=depth+1)

            # Handle pagination
            next_page = PaginationHandler.find_next_page(soup, url)
            if next_page and next_page not in self.visited_listing:
                self.queue.append((next_page, depth))  # Same depth for pagination
                self.graph.add_edge(url, next_page, type="pagination")

            # Progress reporting
            if pages_crawled % 10 == 0:
                print(f"  Progress: {pages_crawled} pages | {len(self.companies)} companies | "
                      f"Queue: {len(self.queue)} | Depth: {depth}")

            # Checkpoint every N companies
            if len(self.companies) - last_checkpoint >= Config.CHECKPOINT_INTERVAL:
                self._save_checkpoint()
                last_checkpoint = len(self.companies)

            sleep_random()

        # Fetch detail pages (parallel)
        if self.visited_detail:
            self._fetch_detail_pages()

        # Finalize stats
        self.stats.end_time = time.time()
        self.stats.avg_response_time = self.client.avg_response_time()
        self.stats.success_rate = self.client.success_rate()

        # Final checkpoint
        self._save_checkpoint()

        print(f"\n✓ Crawl complete!")
        print(f"  Pages: {self.stats.pages_crawled}")
        print(f"  Companies: {self.stats.companies_found}")
        print(f"  Duration: {self.stats.duration()/60:.1f} min")
        print(f"  Speed: {self.stats.pages_per_minute():.1f} pages/min")

        return self.companies, self.stats, self.graph

    def _fetch_detail_pages(self):
        """Fetch detail pages in parallel"""
        print(f"\nFetching {len(self.visited_detail)} detail pages...")

        detail_list = list(self.visited_detail)[:Config.MAX_DETAIL_PAGES]

        def fetch_one(url):
            soup = self.client.get_soup(url)
            if soup:
                return CompanyExtractor.from_detail_page(soup, url)
            return None

        with ThreadPoolExecutor(max_workers=Config.ZYP_DETAIL_WORKERS) as executor:
            futures = {executor.submit(fetch_one, url): url for url in detail_list}

            for i, future in enumerate(as_completed(futures), 1):
                try:
                    company = future.result()
                    if company:
                        self.companies.append(company)
                        self.stats.companies_found += 1
                    self.stats.detail_pages_crawled += 1
                except Exception:
                    self.stats.errors += 1

                if i % 50 == 0:
                    print(f"  Detail pages: {i}/{len(detail_list)} ({self.stats.companies_found} companies)")

# ============================================
# ANALYTICS & VISUALIZATION
# ============================================

class Analytics:
    """Generate statistics and visualizations"""

    @staticmethod
    def generate_summary(all_stats: List[CrawlStats]) -> pd.DataFrame:
        """Create summary statistics DataFrame"""
        summary_data = []

        for stats in all_stats:
            summary_data.append({
                "Seed URL": stats.seed_url,
                "Duration (min)": round(stats.duration() / 60, 2),
                "Pages Crawled": stats.pages_crawled,
                "Detail Pages": stats.detail_pages_crawled,
                "Companies Found": stats.companies_found,
                "Pages/Min": round(stats.pages_per_minute(), 2),
                "Avg Response (s)": round(stats.avg_response_time, 3),
                "Success Rate %": round(stats.success_rate, 1),
                "Errors": stats.errors,
            })

        return pd.DataFrame(summary_data)

    @staticmethod
    def plot_performance(all_stats: List[CrawlStats], output_file: Path):
        """Create performance visualization"""
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle("Scraping Performance Analytics", fontsize=16, fontweight='bold')

        # Extract data
        seeds = [s.seed_url[:30] + "..." for s in all_stats]
        pages = [s.pages_crawled for s in all_stats]
        companies = [s.companies_found for s in all_stats]
        speeds = [s.pages_per_minute() for s in all_stats]
        success_rates = [s.success_rate for s in all_stats]

        # Plot 1: Pages vs Companies
        axes[0, 0].bar(range(len(seeds)), pages, alpha=0.7, label="Pages", color='steelblue')
        axes[0, 0].bar(range(len(seeds)), companies, alpha=0.7, label="Companies", color='coral')
        axes[0, 0].set_xlabel("Seed URL")
        axes[0, 0].set_ylabel("Count")
        axes[0, 0].set_title("Pages Crawled vs Companies Found")
        axes[0, 0].legend()
        axes[0, 0].set_xticks(range(len(seeds)))
        axes[0, 0].set_xticklabels(seeds, rotation=45, ha='right', fontsize=8)

        # Plot 2: Crawl Speed
        axes[0, 1].plot(speeds, marker='o', linewidth=2, markersize=8, color='green')
        axes[0, 1].set_xlabel("Seed Index")
        axes[0, 1].set_ylabel("Pages per Minute")
        axes[0, 1].set_title("Crawl Speed Performance")
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].axhline(sum(speeds)/len(speeds), color='red', linestyle='--',
                           label=f'Avg: {sum(speeds)/len(speeds):.1f}')
        axes[0, 1].legend()

        # Plot 3: Success Rate
        colors = ['green' if sr > 80 else 'orange' if sr > 60 else 'red' for sr in success_rates]
        axes[1, 0].bar(range(len(seeds)), success_rates, color=colors, alpha=0.7)
        axes[1, 0].set_xlabel("Seed URL")
        axes[1, 0].set_ylabel("Success Rate %")
        axes[1, 0].set_title("Request Success Rate")
        axes[1, 0].set_xticks(range(len(seeds)))
        axes[1, 0].set_xticklabels(seeds, rotation=45, ha='right', fontsize=8)
        axes[1, 0].axhline(80, color='green', linestyle='--', alpha=0.5)

        # Plot 4: Efficiency (Companies per Page)
        efficiency = [c/p if p > 0 else 0 for c, p in zip(companies, pages)]
        axes[1, 1].scatter(pages, companies, s=100, alpha=0.6, c=efficiency, cmap='viridis')
        axes[1, 1].set_xlabel("Pages Crawled")
        axes[1, 1].set_ylabel("Companies Found")
        axes[1, 1].set_title("Extraction Efficiency")
        axes[1, 1].grid(True, alpha=0.3)

        # Add trendline
        if len(pages) > 1:
            z = np.polyfit(pages, companies, 1)
            p = np.poly1d(z)
            axes[1, 1].plot(pages, p(pages), "r--", alpha=0.8, linewidth=2)

        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close()

        print(f"✓ Performance plot saved: {output_file}")

    @staticmethod
    def plot_network(graph: nx.DiGraph, seed_url: str, output_file: Path):
        """Visualize BFS crawl network"""
        if len(graph.nodes()) == 0:
            return

        plt.figure(figsize=(16, 12))

        # Layout
        try:
            pos = nx.spring_layout(graph, k=0.5, iterations=50, seed=42)
        except:
            pos = nx.random_layout(graph)

        # Node colors by type
        node_colors = []
        node_sizes = []
        for node in graph.nodes():
            node_type = graph.nodes[node].get('type', 'unknown')
            if node_type == 'seed':
                node_colors.append('red')
                node_sizes.append(500)
            elif node_type == 'listing':
                node_colors.append('lightblue')
                node_sizes.append(200)
            elif node_type == 'detail':
                node_colors.append('lightgreen')
                node_sizes.append(150)
            else:
                node_colors.append('gray')
                node_sizes.append(100)

        # Draw edges by type
        listing_edges = [(u, v) for u, v, d in graph.edges(data=True) if d.get('type') == 'listing']
        detail_edges = [(u, v) for u, v, d in graph.edges(data=True) if d.get('type') == 'detail']
        pagination_edges = [(u, v) for u, v, d in graph.edges(data=True) if d.get('type') == 'pagination']

        nx.draw_networkx_edges(graph, pos, edgelist=listing_edges, edge_color='blue',
                              alpha=0.3, arrows=True, arrowsize=10, width=1)
        nx.draw_networkx_edges(graph, pos, edgelist=detail_edges, edge_color='green',
                              alpha=0.2, arrows=True, arrowsize=8, width=0.5)
        nx.draw_networkx_edges(graph, pos, edgelist=pagination_edges, edge_color='orange',
                              alpha=0.4, arrows=True, arrowsize=10, width=1.5)

        # Draw nodes
        nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=node_sizes, alpha=0.7)

        plt.title(f"BFS Crawl Network Graph\n{seed_url[:60]}...", fontsize=14, fontweight='bold')
        plt.axis('off')

        # Legend
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='red', label='Seed URL'),
            Patch(facecolor='lightblue', label='Listing Pages'),
            Patch(facecolor='lightgreen', label='Detail Pages'),
            Patch(facecolor='blue', alpha=0.3, label='Listing Links'),
            Patch(facecolor='green', alpha=0.3, label='Detail Links'),
            Patch(facecolor='orange', alpha=0.3, label='Pagination'),
        ]
        plt.legend(handles=legend_elements, loc='upper left', fontsize=10)

        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close()

        print(f"✓ Network graph saved: {output_file}")

    @staticmethod
    def plot_data_quality(df: pd.DataFrame, output_file: Path):
        """Visualize data quality metrics"""
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle("Data Quality Analysis", fontsize=16, fontweight='bold')

        # Completeness by field
        completeness = {
            'Name': (df['name'] != '').sum(),
            'Address': (df['address'] != '').sum(),
            'City': (df['city'] != '').sum(),
            'Phone': (df['phone'] != '').sum(),
            'Email': (df['email'] != '').sum(),
            'Website': (df['website'] != '').sum(),
            'Description': (df['description'] != '').sum(),
        }

        axes[0, 0].barh(list(completeness.keys()), list(completeness.values()), color='steelblue')
        axes[0, 0].set_xlabel("Count")
        axes[0, 0].set_title("Field Completeness")
        axes[0, 0].axvline(len(df), color='red', linestyle='--', alpha=0.5)

        # City distribution (top 10)
        city_counts = df['city'].value_counts().head(10)
        axes[0, 1].bar(range(len(city_counts)), city_counts.values, color='coral')
        axes[0, 1].set_xticks(range(len(city_counts)))
        axes[0, 1].set_xticklabels(city_counts.index, rotation=45, ha='right')
        axes[0, 1].set_ylabel("Count")
        axes[0, 1].set_title("Top 10 Cities")

        # Contact information availability
        contact_data = {
            'Phone Only': ((df['phone'] != '') & (df['email'] == '')).sum(),
            'Email Only': ((df['email'] != '') & (df['phone'] == '')).sum(),
            'Both': ((df['phone'] != '') & (df['email'] != '')).sum(),
            'Neither': ((df['phone'] == '') & (df['email'] == '')).sum(),
        }
        axes[1, 0].pie(contact_data.values(), labels=contact_data.keys(), autopct='%1.1f%%',
                       colors=['lightblue', 'lightgreen', 'gold', 'lightcoral'])
        axes[1, 0].set_title("Contact Information Availability")

        # Records per source
        source_counts = df['source_url'].value_counts().head(10)
        axes[1, 1].barh(range(len(source_counts)), source_counts.values, color='green', alpha=0.6)
        axes[1, 1].set_yticks(range(len(source_counts)))
        axes[1, 1].set_yticklabels([url[:40] + "..." for url in source_counts.index], fontsize=8)
        axes[1, 1].set_xlabel("Companies")
        axes[1, 1].set_title("Top 10 Sources")

        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close()

        print(f"✓ Data quality plot saved: {output_file}")

# ============================================
# MAIN ORCHESTRATOR
# ============================================

def extract_all_companies(directories_df: pd.DataFrame, max_workers: int = 2):
    """
    Main function to extract companies from all directories

    Args:
        directories_df: DataFrame with 'listing_urls' column
        max_workers: Number of parallel crawlers

    Returns:
        DataFrame with all extracted companies
    """
    # Filter valid rows
    df = directories_df.copy()
    if "has_listings" in df.columns:
        df = df[df["has_listings"] == True]
    df = df[df["listing_urls"].notna() & (df["listing_urls"].astype(str).str.strip() != "")]

    if len(df) == 0:
        print("❌ No valid listing URLs found")
        return pd.DataFrame()

    # Extract seed URLs
    all_seeds = []
    for _, row in df.iterrows():
        urls = [u.strip() for u in str(row["listing_urls"]).split("|") if u.strip()]
        all_seeds.extend(urls)

    print(f"\n{'='*70}")
    print(f"🚀 ZIMBABWE COMPANY EXTRACTOR")
    print(f"{'='*70}")
    print(f"Total seed URLs: {len(all_seeds)}")
    print(f"Parallel workers: {max_workers}")
    print(f"Checkpoint directory: {Config.CHECKPOINT_DIR}")
    print(f"Output directory: {Config.OUTPUT_DIR}")
    print(f"{'='*70}\n")

    all_companies = []
    all_stats = []
    all_graphs = []

    # Process each seed URL
    for i, seed_url in enumerate(all_seeds, 1):
        print(f"\n[{i}/{len(all_seeds)}] Processing: {seed_url}")

        checkpoint_id = f"seed_{i}_{URLUtils.get_domain(seed_url).replace('.', '_')}"

        try:
            crawler = BFSCrawler(seed_url, checkpoint_id)
            companies, stats, graph = crawler.crawl()

            all_companies.extend(companies)
            all_stats.append(stats)
            all_graphs.append((seed_url, graph))

            # Save incremental results
            if companies:
                temp_df = pd.DataFrame([asdict(c) for c in companies])
                temp_file = Config.OUTPUT_DIR / f"temp_{checkpoint_id}.csv"
                temp_df.to_csv(temp_file, index=False)

        except Exception as e:
            print(f"❌ Error processing {seed_url}: {e}")
            continue

    # Compile final results
    if not all_companies:
        print("\n❌ No companies extracted")
        return pd.DataFrame()

    print(f"\n{'='*70}")
    print(f"📊 GENERATING FINAL RESULTS")
    print(f"{'='*70}\n")

    # Create DataFrame
    final_df = pd.DataFrame([asdict(c) for c in all_companies])

    # Deduplicate
    print(f"Before deduplication: {len(final_df)} records")
    final_df = final_df.drop_duplicates(subset=['name', 'phone', 'website'], keep='first')
    print(f"After deduplication: {len(final_df)} records")

    # Save final CSV
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = Config.OUTPUT_DIR / f"zimbabwe_companies_{timestamp}.csv"
    final_df.to_csv(output_file, index=False)
    print(f"\n✓ Final dataset saved: {output_file}")

    # Generate analytics
    print(f"\n{'='*70}")
    print(f"📈 GENERATING ANALYTICS")
    print(f"{'='*70}\n")

    # Summary statistics
    summary_df = Analytics.generate_summary(all_stats)
    summary_file = Config.STATS_DIR / f"summary_{timestamp}.csv"
    summary_df.to_csv(summary_file, index=False)
    print(f"✓ Summary statistics: {summary_file}")
    print(f"\n{summary_df.to_string()}\n")

    # Performance plots
    perf_plot = Config.STATS_DIR / f"performance_{timestamp}.png"
    Analytics.plot_performance(all_stats, perf_plot)

    # Data quality plots
    quality_plot = Config.STATS_DIR / f"data_quality_{timestamp}.png"
    Analytics.plot_data_quality(final_df, quality_plot)

    # Network graphs (first 5 seeds)
    for i, (seed_url, graph) in enumerate(all_graphs[:5], 1):
        network_file = Config.STATS_DIR / f"network_seed{i}_{timestamp}.png"
        Analytics.plot_network(graph, seed_url, network_file)

    print(f"\n{'='*70}")
    print(f"✅ EXTRACTION COMPLETE")
    print(f"{'='*70}")
    print(f"Total companies: {len(final_df)}")
    print(f"Total pages crawled: {sum(s.pages_crawled for s in all_stats)}")
    print(f"Total duration: {sum(s.duration() for s in all_stats)/60:.1f} minutes")
    print(f"Average speed: {sum(s.pages_per_minute() for s in all_stats)/len(all_stats):.1f} pages/min")
    print(f"{'='*70}\n")

    return final_df

# ============================================
# EXECUTION
# ============================================

if __name__ == "__main__":
    # Load input data
    try:
        directories_with_listings
    except NameError:
        try:
            directories_with_listings = pd.read_csv("directories_with_listings.csv")
            print("✓ Loaded directories_with_listings.csv")
        except Exception as e:
            print(f"❌ Error: {e}")
            print("Please load 'directories_with_listings' DataFrame")
            raise

    # Run extraction
    import numpy as np  # For trendline
    companies_df = extract_all_companies(directories_with_listings, max_workers=2)

    # Display sample
    if not companies_df.empty:
        print("\n" + "="*70)
        print("SAMPLE RESULTS (First 10 records)")
        print("="*70)
        print(companies_df.head(10).to_string())
        print("\n" + "="*70)
        print("FIELD STATISTICS")
        print("="*70)
        print(f"Name:        {(companies_df['name'] != '').sum():>6} / {len(companies_df)}")
        print(f"Address:     {(companies_df['address'] != '').sum():>6} / {len(companies_df)}")
        print(f"City:        {(companies_df['city'] != '').sum():>6} / {len(companies_df)}")
        print(f"Phone:       {(companies_df['phone'] != '').sum():>6} / {len(companies_df)}")
        print(f"Email:       {(companies_df['email'] != '').sum():>6} / {len(companies_df)}")
        print(f"Website:     {(companies_df['website'] != '').sum():>6} / {len(companies_df)}")
        print(f"Description: {(companies_df['description'] != '').sum():>6} / {len(companies_df)}")
        print("="*70 + "\n")


🚀 ZIMBABWE COMPANY EXTRACTOR
Total seed URLs: 7
Parallel workers: 2
Checkpoint directory: checkpoints
Output directory: output


[1/7] Processing: https://afrikta.com/listing-locations/zimbabwe/

Starting BFS crawl: https://afrikta.com/listing-locations/zimbabwe/
Domain: afrikta.com

  Progress: 10 pages | 3 companies | Queue: 120 | Depth: 1
  Progress: 20 pages | 22 companies | Queue: 110 | Depth: 1
  Progress: 30 pages | 39 companies | Queue: 95 | Depth: 1
  Progress: 40 pages | 64 companies | Queue: 82 | Depth: 1
  Progress: 50 pages | 71 companies | Queue: 79 | Depth: 1
  Progress: 60 pages | 71 companies | Queue: 79 | Depth: 1
  Progress: 70 pages | 71 companies | Queue: 79 | Depth: 1
  Progress: 80 pages | 71 companies | Queue: 65 | Depth: 1
  Progress: 90 pages | 150 companies | Queue: 51 | Depth: 1
  Progress: 100 pages | 167 companies | Queue: 50 | Depth: 1
  Progress: 110 pages | 167 companies | Queue: 50 | Depth: 1
  Progress: 120 pages | 167 companies | Queue: 50 | Depth: 

### Visuals